# Pre-Pipeline: Import and combine dataset

In [10]:
import pandas as pd
df1 = pd.read_csv('original_dataset/hellosehat_dataset_10.csv', sep=';', encoding='utf-8')
df2 = pd.read_csv('original_dataset/alodokter_dataset_10.csv', sep=';', encoding='utf-8')
df3 = pd.read_csv('original_dataset/doktersehat_gizi_final_10.csv', sep=';', encoding='utf-8')

In [11]:
df_combine = pd.concat([df1, df2, df3], ignore_index=True)
print(f"Total dokumen setelah concat: {len(df_combine)}")
print(f"Index range: {df_combine.index.min()} - {df_combine.index.max()}")
print(f"Kolom: {df_combine.columns.tolist()}")

Total dokumen setelah concat: 2391
Index range: 0 - 2390
Kolom: ['URL', 'Judul', 'Konten']


In [13]:
df_combine.head()

,URL,Judul,Konten
0,https://hellosehat.com/nutrisi/berat-badan-tur...,Kenali 9 Penyebab Perut Buncit dan Cara Mengat...,Perut buncit memang mampu memengaruhi penampil...
1,https://hellosehat.com/nutrisi/tips-makan-seha...,"8 Merk Oven Gas Terbaik, Cocok untuk Bisnis Kue","Bagi Anda yang gemar bikin kue, oven gas menja..."
2,https://hellosehat.com/nutrisi/fakta-gizi/mere...,10 Merek Oatmeal yang Bergizi dan Cocok untuk ...,Butuh menu sarapan yang cepat? Berbagai merek ...
3,https://hellosehat.com/nutrisi/resep-sehat/jus...,4 Resep Jus untuk Bantu Meningkatkan Sistem Im...,"Setiap harinya, sistem imunitas pada tubuh bek..."
4,https://hellosehat.com/nutrisi/berat-badan-tur...,Apakah Sering Buang Air Bisa Menurunkan Berat ...,"Setelah diolah, dicerna, dan diambil semua giz..."


In [12]:
# FIX that thang!
# df_combine.to_csv('output_dataset/combined_nutrition_dataset_2.csv', index=False, encoding='utf-8', sep=";")

In [25]:
df = pd.read_csv('output_dataset/combined_nutrition_dataset.csv', sep=';', encoding='utf-8')
df.head()

,URL,Judul,Konten
0,https://hellosehat.com/nutrisi/berat-badan-tur...,Kenali 9 Penyebab Perut Buncit dan Cara Mengat...,Perut buncit memang mampu memengaruhi penampil...
1,https://hellosehat.com/nutrisi/tips-makan-seha...,"8 Merk Oven Gas Terbaik, Cocok untuk Bisnis Kue","Bagi Anda yang gemar bikin kue, oven gas menja..."
2,https://hellosehat.com/nutrisi/fakta-gizi/mere...,10 Merek Oatmeal yang Bergizi dan Cocok untuk ...,Butuh menu sarapan yang cepat? Berbagai merek ...
3,https://hellosehat.com/nutrisi/resep-sehat/jus...,4 Resep Jus untuk Bantu Meningkatkan Sistem Im...,"Setiap harinya, sistem imunitas pada tubuh bek..."
4,https://hellosehat.com/nutrisi/berat-badan-tur...,Apakah Sering Buang Air Bisa Menurunkan Berat ...,"Setelah diolah, dicerna, dan diambil semua giz..."


In [26]:
row = df.loc[2150]
print(row)

URL       https://doktersehat.com/gaya-hidup/gizi-dan-nu...
Judul        Sumber, Manfaat, dan Dampak Kekurangan Omega-3
Konten     Kita selalu menganggap kalau omega-3 adalah n...
Name: 2150, dtype: object


# Pre-Pipeline: Intent Tagging (Health Goals)

In [27]:
import pandas as pd
import csv
import sys

In [28]:
# --- 1. KONFIGURASI INPUT & OUTPUT ---
INPUT_FILE = 'output_dataset/combined_nutrition_dataset.csv'
OUTPUT_FILE = 'output_dataset/tagged_combined_nutrition_dataset.csv'


In [29]:

# --- 2. KAMUS KEYWORD INTENT ---
INTENT_KEYWORDS = {
    'diabetes': ['diabetes', 'gula darah', 'kencing manis', 'insulin', 'glukosa', 'hiperglikemia'],
    'anemia': ['anemia', 'kurang darah', 'zat besi', 'hemoglobin', 'pucat', 'lelah'],
    'kesehatan_ibu': ['hamil', 'menyusui', 'bumil', 'busui', 'asi', 'janin', 'kandungan', 'kehamilan'],
    'kesehatan_anak': ['anak', 'bayi', 'balita', 'si kecil', 'tumbuh kembang', 'imunisasi'],
    'berat_badan': ['berat badan', 'diet', 'kurus', 'gemuk', 'langsing', 'turun berat', 'buncit', 'lemak', 'kalori', 'obesitas'],
    'pembentukan_tubuh': ['otot', 'gym', 'fitness', 'binaraga', 'sixpack', 'latihan beban', 'workout', 'massa otot'],
    'kesehatan_pencernaan': ['pencernaan', 'usus', 'lambung', 'maag', 'gerd', 'sembelit', 'diare', 'serat'],
    'resep_sehat': ['resep', 'cara membuat', 'bahan-bahan', 'menu masakan', 'cara masak', 'hidangan'],
    'diet_khusus': ['keto', 'vegan', 'vegetarian', 'gluten free', 'rendah garam', 'dash diet', 'intermittent'],
    'pencegahan': ['mencegah', 'risiko', 'hindari', 'bahaya', 'waspada', 'gejala', 'tanda-tanda'],
    'fakta_gizi': ['kandungan gizi', 'nutrisi', 'protein', 'karbohidrat', 'vitamin', 'mineral', 'takaran saji'],
    'makanan_sehat': ['buah', 'sayur', 'organik', 'superfood', 'makanan sehat', 'bijian', 'kacang'],
    # Fallback
    'Kesehatan_umum': ['manfaat', 'khasiat', 'sehat', 'bugar', 'stamina', 'daya tahan', 'imun', 'kesehatan']
}



In [30]:
def get_top_3_intents(text):
    """
    Menghitung frekuensi keyword dan mengembalikan maksimal 3 intent teratas.
    """
    text = str(text).lower()
    scores = {}
    
    for intent, keywords in INTENT_KEYWORDS.items():
        count = 0
        for kw in keywords:
            count += text.count(kw)
        if count > 0:
            scores[intent] = count
    
    # Jika tidak ada match, return default
    if not scores:
        return "Kesehatan_umum"
    
    # Urutkan score tertinggi -> terendah
    sorted_intents = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    
    # Ambil 3 teratas
    top_3 = [item[0] for item in sorted_intents[:3]]
    return ", ".join(top_3)



In [31]:
# --- 3. EKSEKUSI UTAMA ---
def process_tagging():
    print(f"📂 Membaca file input: {INPUT_FILE} ...")
    
    try:
        # Membaca file dengan separator ';'
        # Kita asumsikan baris pertama adalah header [URL;Judul;Konten]
        df = pd.read_csv(INPUT_FILE, sep=';', on_bad_lines='skip')
        
        # Validasi kolom dasar (Sesuaikan nama kolom jika di file Anda berbeda)
        # Kita rename standar agar mudah diproses
        # Asumsi urutan kolom: 1. URL, 2. Judul, 3. Konten
        if len(df.columns) >= 3:
            df.columns = ['url', 'title', 'content'] + list(df.columns[3:])
        else:
            print("❌ Error: File input harus memiliki minimal 3 kolom (URL;Judul;Konten)")
            return

        print(f"📊 Total Data: {len(df)} baris.")
        
        # Bersihkan data (isi yang kosong dengan string kosong)
        df['title'] = df['title'].fillna('').astype(str)
        df['content'] = df['content'].fillna('').astype(str)
        
        print("🔍 Melakukan Auto-Tagging Intent...")
        # Gabungkan Judul + Konten untuk pencarian keyword yang lebih akurat
        df['full_text_scan'] = df['title'] + " " + df['content']
        df['intent'] = df['full_text_scan'].apply(get_top_3_intents)
        
        # Hapus kolom bantuan
        df.drop(columns=['full_text_scan'], inplace=True)
        
        print(f"💾 Menyimpan ke format final (Comma Separated + Quoted)...")
        
        # --- TEKNIK PENYIMPANAN PENTING ---
        # quoting=csv.QUOTE_ALL : Memaksa SEMUA kolom dibungkus tanda kutip "..."
        # Ini menjamin koma di dalam teks TIDAK akan dianggap sebagai pemisah kolom baru.
        df.to_csv(
            OUTPUT_FILE, 
            sep=',',              # Separator Koma
            quotechar='"',        # Pembungkus Tanda Kutip Ganda
            quoting=csv.QUOTE_ALL, # MODE AMAN: Bungkus semua data dengan kutip
            index=False,
            encoding='utf-8'
        )
        
        print(f"✅ SUKSES! File tersimpan: {OUTPUT_FILE}")
        print("   Format: \"URL\",\"Title\",\"Content\",\"Intent\" (Comma Separated)")

    except FileNotFoundError:
        print(f"❌ File {INPUT_FILE} tidak ditemukan.")
    except Exception as e:
        print(f"❌ Terjadi kesalahan: {e}")

if __name__ == "__main__":
    process_tagging()

📂 Membaca file input: output_dataset/combined_nutrition_dataset.csv ...
📊 Total Data: 2425 baris.
🔍 Melakukan Auto-Tagging Intent...
💾 Menyimpan ke format final (Comma Separated + Quoted)...
✅ SUKSES! File tersimpan: output_dataset/tagged_combined_nutrition_dataset.csv
   Format: "URL","Title","Content","Intent" (Comma Separated)


# Tahap 1: Preprocessing and chunking 

In [1]:
import pandas as pd
import re
from transformers import BertTokenizer
import json


KeyboardInterrupt



In [ ]:
# Load dataset
df = pd.read_csv('output_dataset/tagged_combined_nutrition_dataset.csv')
print(f"Total dokumen: {len(df)}")
print(f"Kolom dataset: {df.columns.tolist()}")

In [ ]:
# Load IndoBERT tokenizer
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p2')

# Fungsi preprocessing
def preprocess_text(text):
    """Normalisasi teks"""
    if pd.isna(text):
        return ""
    
    text = text.lower()
    text = re.sub(r'[^\w\s.,!?%-]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    
    return text

# Fungsi chunking dengan sliding window
def chunk_text_with_overlap(text, max_tokens=384, overlap=50):
    """
    Chunking dengan overlap sliding window
    max_tokens: 512 - 128 (buffer untuk question) = 384
    overlap: 50 tokens
    """
    chunks = []
    tokens = tokenizer.tokenize(text)
    
    if len(tokens) <= max_tokens:
        return [text]
    
    start = 0
    while start < len(tokens):
        end = min(start + max_tokens, len(tokens))
        chunk_tokens = tokens[start:end]
        chunk_text = tokenizer.convert_tokens_to_string(chunk_tokens)
        chunks.append(chunk_text)
        
        if end == len(tokens):
            break
        
        start = end - overlap
    
    return chunks

In [ ]:
# Proses semua dokumen
processed_data = []

for idx, row in df.iterrows():
    title = row.get('title', '')
    content = row.get('content', '')
    url = row.get('url', '')
    intent = row.get('intent', '')
    
    full_text = f"{title}. {content}".strip()
    cleaned_text = preprocess_text(full_text)
    
    if not cleaned_text:
        continue
    
    chunks = chunk_text_with_overlap(cleaned_text)
    
    for chunk_idx, chunk in enumerate(chunks):
        processed_data.append({
            'doc_id': idx,
            'chunk_id': chunk_idx,
            'title': title,
            'text': chunk,
            'url': url,
            'intent': intent,
            'token_count': len(tokenizer.tokenize(chunk))
        })
    
    if (idx + 1) % 100 == 0:
        print(f"Processed {idx + 1}/{len(df)} documents")

# Simpan hasil
df_processed = pd.DataFrame(processed_data)
df_processed.to_csv('output_dataset/processed_chunks.csv', index=False)

print(f"\nTotal chunks: {len(df_processed)}")
print(f"Rata-rata tokens per chunk: {df_processed['token_count'].mean():.2f}")
print(f"Max tokens: {df_processed['token_count'].max()}")
print(f"Min tokens: {df_processed['token_count'].min()}")

# Simpan corpus untuk MLM
all_texts = df_processed['text'].tolist()
with open('corpus_for_mlm.json', 'w', encoding='utf-8') as f:
    json.dump(all_texts, f, ensure_ascii=False, indent=2)

print("\nFile tersimpan:")
print("- processed_chunks.csv")
print("- corpus_for_mlm.json")

# Tahap 2: Domain ~~EXPANSION~~ Adaptation

In [ ]:
import json
import torch
import re
import os
import gc
from transformers import (
    BertTokenizer, 
    BertForMaskedLM,
    DataCollatorForWholeWordMask,
    TrainingArguments,
    Trainer
)
from torch.utils.data import Dataset
from tqdm.auto import tqdm

In [2]:
# --- 1. SETUP GPU & CLEAR MEMORY ---
# Cek apakah CUDA (GPU) tersedia
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"✅ GPU Ditemukan: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM Total: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    
    # Bersihkan cache memori sebelumnya (Penting jika menggunakan Notebook)
    torch.cuda.empty_cache()
    gc.collect()
else:
    print("GPU tidak ditemukan! Menggunakan CPU")
    device = torch.device("cpu")

✅ GPU Ditemukan: NVIDIA GeForce RTX 4050 Laptop GPU
   VRAM Total: 6.44 GB


In [3]:
# --- 2. LOAD DATA ---
# Load corpus
try:
    with open('corpus_for_mlm.json', 'r', encoding='utf-8') as f:
        all_texts = json.load(f)
    print(f"Total texts: {len(all_texts)}")
except FileNotFoundError:
    print("File corpus tidak ditemukan, menggunakan dummy data.")
    all_texts = ["1", "2"]

# SPLIT DATA (90% Train, 10% Validation)
# Kita butuh ini agar model bisa diuji (Val Loss muncul)
split_idx = int(0.9 * len(all_texts))
train_texts = all_texts[:split_idx]
val_texts = all_texts[split_idx:]

print(f"Data Training: {len(train_texts)}")
print(f"Data Validasi: {len(val_texts)}")

# Load model dan tokenizer
model_name = 'indobenchmark/indobert-base-p2'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

Total texts: 7661
Data Training: 6894
Data Validasi: 767


Some weights of BertForMaskedLM were not initialized from the model checkpoint at indobenchmark/indobert-base-p2 and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# --- 3. PINDAHKAN MODEL KE GPU ---
model.to(device)

# Custom Dataset untuk MLM
class MLMDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=512):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length
        
        # Preprocess dengan progress bar
        print("Preprocessing texts...")
        self.encodings = []
        for text in tqdm(texts, desc="Tokenizing"):
            encoding = self.tokenizer(
                text,
                truncation=True,
                max_length=self.max_length,
                padding='max_length',
                return_tensors='pt'
            )
            self.encodings.append({
                'input_ids': encoding['input_ids'].squeeze(),
                'attention_mask': encoding['attention_mask'].squeeze()
            })
    
    def __len__(self):
        return len(self.encodings)
    
    def __getitem__(self, idx):
        return self.encodings[idx]

# Custom Data Collator
class CustomWWMDataCollator(DataCollatorForWholeWordMask):
    def __init__(self, tokenizer, mlm_probability=0.15):
        super().__init__(
            tokenizer=tokenizer,
            mlm=True,
            mlm_probability=mlm_probability
        )
    
    def torch_mask_tokens(self, inputs, special_tokens_mask=None):
        labels = inputs.clone()
        probability_matrix = torch.full(labels.shape, self.mlm_probability)
        
        if special_tokens_mask is None:
            special_tokens_mask = [
                self.tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True)
                for val in labels.tolist()
            ]
            special_tokens_mask = torch.tensor(special_tokens_mask, dtype=torch.bool)
        else:
            special_tokens_mask = special_tokens_mask.bool()
        
        probability_matrix.masked_fill_(special_tokens_mask, value=0.0)
        
        for i, input_ids in enumerate(inputs):
            tokens = self.tokenizer.convert_ids_to_tokens(input_ids)
            for j, token in enumerate(tokens):
                if re.search(r'\d', token) or token in ['mg', 'gram', 'kg', 'ml', 'kkal', 'kalori', '%']:
                    probability_matrix[i, j] = 0.0
        
        masked_indices = torch.bernoulli(probability_matrix).bool()
        labels[~masked_indices] = -100
        indices_replaced = torch.bernoulli(torch.full(labels.shape, 0.8)).bool() & masked_indices
        inputs[indices_replaced] = self.tokenizer.convert_tokens_to_ids(self.tokenizer.mask_token)
        indices_random = torch.bernoulli(torch.full(labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced
        random_words = torch.randint(len(self.tokenizer), labels.shape, dtype=torch.long)
        inputs[indices_random] = random_words[indices_random]
        return inputs, labels

# Buat 2 Dataset Terpisah
print("\nMempersiapkan Training Set...")
train_dataset = MLMDataset(train_texts, tokenizer)

print("Mempersiapkan Validation Set...")
val_dataset = MLMDataset(val_texts, tokenizer)

# Data collator
data_collator = CustomWWMDataCollator(
    tokenizer=tokenizer,
    mlm_probability=0.15
)


Mempersiapkan Training Set...
Preprocessing texts...


Tokenizing:   0%|          | 0/6894 [00:00<?, ?it/s]

Mempersiapkan Validation Set...
Preprocessing texts...


Tokenizing:   0%|          | 0/767 [00:00<?, ?it/s]

C:\Users\LENOVO\AppData\Local\Programs\Python\Python313\Lib\site-packages\transformers\data\data_collator.py:1642: FutureWarning: DataCollatorForWholeWordMask is deprecated and will be removed in a future version, you can now use DataCollatorForLanguageModeling with whole_word_mask=True instead.
  warnings.warn(


In [6]:
# --- 4. TRAINING ARGUMENTS ---
training_args = TrainingArguments(
    output_dir='./indobert-gizi-mlm',
    overwrite_output_dir=True,
      
    # === HYPERPARAMETERS ===
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,              
    learning_rate=2e-5,
    weight_decay=0.01,

    # === EVALUATION SETTINGS (BARU) ===
    eval_strategy="steps",
    eval_steps=100,
    save_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    
    # === LOGGING ===
    logging_dir='./logs',
    logging_steps=100,
    
    # === SYSTEM ===
    fp16=torch.cuda.is_available(),
    no_cuda=False,
    dataloader_pin_memory=True,
    dataloader_num_workers=0,       
    disable_tqdm=False,
    report_to='none',
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

print("\nMulai Training...")
trainer.train()

# Simpan Model Akhir
print("\nMenyimpan model...")
model.save_pretrained("./indobert-gizi-mlm-final")
tokenizer.save_pretrained("./indobert-gizi-mlm-final")
print("Selesai!")


Mulai Training...


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Step,Training Loss,Validation Loss
100,5.563700,4.428384
200,4.044200,3.764034
300,3.597000,3.424848
400,3.361500,3.217364
500,3.164900,3.068305
600,3.013400,2.934664
700,2.841500,2.784796
800,2.731400,2.662641
900,2.627900,2.553934
1000,2.576000,2.442628


There were missing keys in the checkpoint model loaded: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias'].



Menyimpan model...
Selesai!


## Clear Garbage

In [7]:
import torch
import gc 

def clear_gpu_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()

clear_gpu_memory()

# Tahap 3: Membuat Pasangan QA

In [1]:
import pandas as pd
import json
import uuid
import time
import os
from tqdm import tqdm
from groq import Groq, RateLimitError

In [2]:
# --- 1. KONFIGURASI ---
API_KEY = "apikey" 
client = Groq(api_key=API_KEY)

# Model
PRIMARY_MODEL = "llama-3.3-70b-versatile"

# File Paths (Sesuaikan dengan notebook Anda)
INPUT_FILE = 'output_dataset/dataset_filtered_processed_chunks.csv'
OUTPUT_FILE = '3_qa_dataset_strict_gold.jsonl'

In [3]:
# --- 2. FUNGSI GENERATOR STRICT ---
def generate_qa_strict(context):
    prompt = f"""
    You are a strict Data Annotator for a SQuAD dataset.
    
    TASK: Extract 3 QA pairs from the provided text.
    
    CRITICAL RULES:
    1. The "answer" MUST be an EXACT SUBSTRING from the Context. Copy-paste ONLY. No paraphrasing.
    2. Provide 2 answerable questions and 1 unanswerable question.
    3. Language: Indonesian.
    
    CONTEXT:
    "{context}"
    
    OUTPUT FORMAT (JSON ONLY):
    [
      {{ "question": "Pertanyaan?", "answer": "exact substring", "is_impossible": false }},
      {{ "question": "Pertanyaan sulit?", "answer": "", "is_impossible": true }}
    ]
    """
    
    max_retries = 5
    retry_count = 0

    while retry_count < max_retries:
        try:
            completion = client.chat.completions.create(
                messages=[
                    {"role": "system", "content": "You output JSON only."},
                    {"role": "user", "content": prompt}
                ],
                model=PRIMARY_MODEL,
                temperature=0.0, # Wajib 0 agar tidak kreatif (konsisten)
                response_format={"type": "json_object"}
            )
            return json.loads(completion.choices[0].message.content)

        except RateLimitError as e:
            # --- LOGIKA MENANGANI RATE LIMIT (429) ---
            print(f"⚠️ Rate Limit Hit! (Percobaan {retry_count + 1}/{max_retries})")
            
            # Coba ambil informasi waktu tunggu dari header response
            retry_after = 0
            try:
                headers = e.response.headers
                
                # Coba header standar 'retry-after'
                if 'retry-after' in headers:
                    retry_after = float(headers['retry-after'])
                
                # Coba header spesifik 'x-ratelimit-reset-tokens' (biasanya format "12.34s")
                elif 'x-ratelimit-reset-tokens' in headers:
                    reset_tokens = headers['x-ratelimit-reset-tokens']
                    retry_after = float(reset_tokens.replace('s', ''))
                
                # Coba header spesifik 'x-ratelimit-reset-requests'
                elif 'x-ratelimit-reset-requests' in headers:
                    reset_reqs = headers['x-ratelimit-reset-requests']
                    # Parse format "12m30s" atau "12.34s" jika perlu, tapi float simpel biasanya cukup untuk seconds
                    if 'm' in reset_reqs: 
                        # Sederhana: jika ada menit, tunggu default 60 detik saja biar aman
                        retry_after = 60 
                    else:
                        retry_after = float(reset_reqs.replace('s', ''))
            
            except Exception as header_err:
                # Jika gagal parsing header, gunakan default
                print(f"   (Gagal baca header: {header_err})")
                retry_after = 20 # Default aman

            # Pastikan retry_after minimal 1 detik & tambahkan buffer
            wait_time = max(1, retry_after) + 2 
            
            print(f"⏳ Tidur selama {wait_time:.2f} detik sebelum mencoba lagi...")
            time.sleep(wait_time)
            
            retry_count += 1
            
        except Exception as e:
            # Error lain (misal koneksi putus, JSON malformed dari server, dll)
            print(f"❌ Error non-limit: {e}")
            return None

    print("❌ Gagal setelah max retries.")
    return None

In [4]:
def run_strict_pipeline():
    print(f"⚙️ Menyiapkan Pipeline Strict Mode dengan Model: {PRIMARY_MODEL}")
    
    # Baca Data Input
    try:
        df = pd.read_csv(INPUT_FILE)
        # Ambil kolom text, pastikan string
        texts = df['text'].fillna('').astype(str).unique()
        total_awal = len(texts)
    except Exception as e:
        print(f"❌ Gagal baca input: {e}")
        return

    # Logika Resume (Cek data yg sudah ada)
    processed_contexts = set()
    if os.path.exists(OUTPUT_FILE):
        print(f"📂 Membaca checkpoint dari {OUTPUT_FILE}...")
        with open(OUTPUT_FILE, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    data = json.loads(line)
                    processed_contexts.add(data['context'])
                except: continue
    
    # Filter data yang belum diproses
    texts_to_process = [t for t in texts if t not in processed_contexts]
    print(f"📊 Sisa data: {len(texts_to_process)} dari {total_awal} total chunk unik.")
    
    if not texts_to_process:
        print("🎉 Semua data sudah selesai!")
        return

    valid_count = 0
    dropped_count = 0
    
    # Loop Utama
    with open(OUTPUT_FILE, 'a', encoding='utf-8') as f:
        for text in tqdm(texts_to_process, desc="Generating QA"):
            if len(text) < 50: continue
            
            # Request ke LLM (Pakai fungsi SMART)
            qa_list = generate_qa_strict(text)
            
            # Parsing response
            if qa_list:
                # Handle variasi struktur JSON dari LLM
                if isinstance(qa_list, dict): 
                    qa_list = qa_list.get('qas', qa_list.get('qa_pairs', []))
                
                squad_qas = []
                # Pastikan qa_list adalah list sebelum iterasi
                if isinstance(qa_list, list):
                    for item in qa_list:
                        # Validasi Format
                        if not isinstance(item, dict): continue
                        
                        question = item.get('question')
                        answer_text = item.get('answer', '')
                        is_impossible = item.get('is_impossible', False)
                        
                        if is_impossible:
                            squad_qas.append({
                                "id": str(uuid.uuid4()),
                                "question": question,
                                "answers": [],
                                "is_impossible": True
                            })
                        else:
                            # --- STRICT CHECK (Validasi Substring) ---
                            if not answer_text: continue
                            
                            # Case insensitive search untuk fleksibilitas sedikit
                            # Tapi kita simpan text ASLI dari konteks
                            start_idx = text.find(answer_text)
                            
                            if start_idx != -1:
                                # LOLOS VALIDASI
                                squad_qas.append({
                                    "id": str(uuid.uuid4()),
                                    "question": question,
                                    "answers": [{
                                        "text": answer_text,
                                        "answer_start": start_idx
                                    }],
                                    "is_impossible": False
                                })
                                valid_count += 1
                            else:
                                # GAGAL VALIDASI (Halusinasi LLM) -> BUANG
                                dropped_count += 1
                
                # Simpan jika ada QA valid
                if squad_qas:
                    entry = {"context": text, "qas": squad_qas}
                    f.write(json.dumps(entry) + "\n")
                    f.flush()
            

    print("\n" + "="*50)
    print("🏁 PROSES SELESAI")
    print(f"✅ QA Valid Tersimpan: {valid_count}")
    print(f"🗑️ QA Halusinasi Dibuang: {dropped_count}")
    print(f"📂 Output: {OUTPUT_FILE}")
    print("="*50)

In [5]:
# Jalankan
if __name__ == "__main__":
    run_strict_pipeline()

⚙️ Menyiapkan Pipeline Strict Mode dengan Model: llama-3.3-70b-versatile
📂 Membaca checkpoint dari 3_qa_dataset_strict_gold.jsonl...
📊 Sisa data: 38 dari 752 total chunk unik.


Generating QA: 100%|███████████████████████████████████████████████████████████████████| 38/38 [02:08<00:00,  3.39s/it]


🏁 PROSES SELESAI
✅ QA Valid Tersimpan: 56
🗑️ QA Halusinasi Dibuang: 20
📂 Output: 3_qa_dataset_strict_gold.jsonl


# Tahap 4: Fine-Tuning QA

In [1]:
import json
import torch
import gc
from transformers import (
    BertTokenizerFast,
    BertForQuestionAnswering,
    TrainingArguments,
    Trainer,
    default_data_collator,
)
from torch.utils.data import Dataset
from tqdm.auto import tqdm
from transformers import BertModel, BertConfig 


In [2]:

# Clear GPU memory
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ GPU tidak tersedia, menggunakan CPU")


✅ GPU: NVIDIA GeForce RTX 4050 Laptop GPU
   VRAM: 6.44 GB


In [3]:
qa_pairs = []
with open('3_qa_dataset_strict_gold.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        qa_pairs.append(json.loads(line))

print(f"Total paragraf: {len(qa_pairs)}")

Total paragraf: 752


In [4]:
# Load model dari Domain Adaptation
model_path = './indobert-gizi-mlm-final'
tokenizer = BertTokenizerFast.from_pretrained(model_path)

# Load base BERT dari MLM model, lalu inisialisasi QA head baru
from transformers import BertConfig, BertForQuestionAnswering

print("\n🔧 Loading BERT encoder dari Domain Adaptation...")

# === PERBAIKAN DI SINI ===
# Kita paksa num_labels=2 agar output layer QA hanya 2 (Start & End)
config = BertConfig.from_pretrained(model_path, num_labels=2)

# Gunakan .from_pretrained langsung dengan ignore_mismatched_sizes=True
# Ini akan memuat body BERT yang sudah di-training (MLM), tapi mereset head QA menjadi benar
model = BertForQuestionAnswering.from_pretrained(
    model_path, 
    config=config,
    ignore_mismatched_sizes=True
)

print("✅ Model QA berhasil diinisialisasi.")
print(f"🔍 Cek QA Outputs Layer: {model.qa_outputs}") 
# Pastikan output print di atas tertulis out_features=2

model.to('cuda' if torch.cuda.is_available() else 'cpu')


🔧 Loading BERT encoder dari Domain Adaptation...


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at ./indobert-gizi-mlm-final and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Model QA berhasil diinisialisasi.
🔍 Cek QA Outputs Layer: Linear(in_features=768, out_features=2, bias=True)


BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [5]:
# Parse data ke format training
train_examples = []

for para in qa_pairs:
    context = para['context']
    for qa in para['qas']:
        example = {
            'id': qa['id'],
            'question': qa['question'],
            'context': context,
        }
        
        if qa['is_impossible']:
            example['answers'] = {'answer_start': [], 'text': []}
        else:
            example['answers'] = {
                'answer_start': [qa['answers'][0]['answer_start']],
                'text': [qa['answers'][0]['text']]
            }
        
        train_examples.append(example)

print(f"Total QA pairs: {len(train_examples)}")



Total QA pairs: 1911


In [6]:
# Split train/validation (90/10)
split_idx = int(0.9 * len(train_examples))
train_data = train_examples[:split_idx]
val_data = train_examples[split_idx:]

print(f"Training: {len(train_data)}")
print(f"Validation: {len(val_data)}")


Training: 1719
Validation: 192


In [7]:
# Tokenisasi dengan batch processing
def tokenize_dataset(examples):
    questions = [ex['question'] for ex in examples]
    contexts = [ex['context'] for ex in examples]
    
    tokenized = tokenizer(
        questions,
        contexts,
        truncation='only_second',
        max_length=512,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding='max_length'
    )
    
    sample_mapping = tokenized.pop('overflow_to_sample_mapping')
    offset_mapping = tokenized['offset_mapping']
    
    start_positions = []
    end_positions = []
    
    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized['input_ids'][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        sequence_ids = tokenized.sequence_ids(i)
        sample_index = sample_mapping[i]
        answers = examples[sample_index]['answers']
        
        # Unanswerable question
        if len(answers['answer_start']) == 0:
            start_positions.append(cls_index)
            end_positions.append(cls_index)
            continue
        
        start_char = answers['answer_start'][0]
        end_char = start_char + len(answers['text'][0])
        
        # Find token start position
        token_start_index = 0
        while token_start_index < len(sequence_ids) and sequence_ids[token_start_index] != 1:
            token_start_index += 1
        
        # Find token end position
        token_end_index = len(input_ids) - 1
        while token_end_index >= 0 and sequence_ids[token_end_index] != 1:
            token_end_index -= 1
        
        # Check if answer is in this chunk
        if not (token_start_index < len(offsets) and 
                token_end_index < len(offsets) and
                offsets[token_start_index][0] <= start_char and 
                offsets[token_end_index][1] >= end_char):
            start_positions.append(cls_index)
            end_positions.append(cls_index)
            continue
        
        # Find exact token positions
        while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
            token_start_index += 1
        start_positions.append(token_start_index - 1)
        
        while token_end_index >= 0 and offsets[token_end_index][1] >= end_char:
            token_end_index -= 1
        end_positions.append(token_end_index + 1)
    
    tokenized['start_positions'] = start_positions
    tokenized['end_positions'] = end_positions
    tokenized.pop('offset_mapping')
    
    return tokenized

print("\n🔄 Tokenizing training data...")
tokenized_train = tokenize_dataset(train_data)

print("🔄 Tokenizing validation data...")
tokenized_val = tokenize_dataset(val_data)



🔄 Tokenizing training data...
🔄 Tokenizing validation data...


In [8]:
# Dataset class
class QADataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    
    def __len__(self):
        return len(self.encodings['input_ids'])
    
    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.encodings['input_ids'][idx]),
            'attention_mask': torch.tensor(self.encodings['attention_mask'][idx]),
            'start_positions': torch.tensor(self.encodings['start_positions'][idx]),
            'end_positions': torch.tensor(self.encodings['end_positions'][idx])
        }

train_dataset = QADataset(tokenized_train)
val_dataset = QADataset(tokenized_val)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Val dataset size: {len(val_dataset)}")

Train dataset size: 1719
Val dataset size: 192


In [9]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./indobert-gizi-qa',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=3e-5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    save_steps=100,
    save_total_limit=2,
    eval_strategy='steps',
    eval_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model='loss',
    fp16=torch.cuda.is_available(),
    dataloader_num_workers=0,
    disable_tqdm=False,
    report_to='none',
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=default_data_collator,
)


In [10]:
# Training
print("\n" + "="*50)
print("Memulai Fine-Tuning QA...")
print("="*50)
trainer.train()

# Simpan model
print("\n💾 Menyimpan model...")
model.save_pretrained('./indobert-gizi-qa-final')
tokenizer.save_pretrained('./indobert-gizi-qa-final')

print("\n" + "="*50)
print("Fine-Tuning QA selesai!")
print("Model tersimpan di: ./indobert-gizi-qa-final")
print("="*50)


Memulai Fine-Tuning QA...


Step,Training Loss,Validation Loss
100,2.953500,2.694164
200,2.577300,2.419097
300,1.803400,2.265416
400,1.559100,2.194084
500,1.059200,2.297399
600,1.013200,2.223674



💾 Menyimpan model...

Fine-Tuning QA selesai!
Model tersimpan di: ./indobert-gizi-qa-final


# Test Model

In [29]:
from transformers import BertTokenizerFast, BertForQuestionAnswering, pipeline

# 1. Load Model Final
model_path = "./indobert-gizi-qa-final" # Pastikan path ini sesuai output training
tokenizer = BertTokenizerFast.from_pretrained(model_path)
model = BertForQuestionAnswering.from_pretrained(model_path)

# 2. Buat Pipeline QA
qa_pipeline = pipeline(
    "question-answering",
    model=model,
    tokenizer=tokenizer,
    device=0 # Gunakan GPU (0) atau CPU (-1)
)

# 3. Tes Manual
context_text = """
Diabetes melitus adalah penyakit kronis yang ditandai dengan kadar gula darah yang tinggi. 
Penyebab utamanya adalah kurangnya produksi insulin oleh pankreas atau ketidakmampuan tubuh menggunakan insulin.
Gejala umum meliputi sering haus, sering buang air kecil, dan penurunan berat badan drastis.
Pencegahan dapat dilakukan dengan menjaga pola makan sehat dan rutin berolahraga.
"""

questions = [
    "Apa itu diabetes melitus?",
    "Apa penyebab utama diabetes?",
    "Bagaimana cara mencegahnya diabetes melitus?",
    "Apa gejala Diabetes melitus?"
]

print("="*50)
print("🤖 HASIL TES MODEL QA")
print("="*50)

for q in questions:
    result = qa_pipeline(question=q, context=context_text)
    print(f"Tanya : {q}")
    print(f"Jawab : {result['answer']}")
    print(f"Score : {result['score']:.4f}")
    print("-" * 30)

Device set to use cuda:0


🤖 HASIL TES MODEL QA
Tanya : Apa itu diabetes melitus?
Jawab : kurangnya produksi insulin oleh pankreas atau ketidakmampuan tubuh menggunakan insulin
Score : 0.1257
------------------------------
Tanya : Apa penyebab utama diabetes?
Jawab : kurangnya produksi insulin oleh pankreas atau ketidakmampuan tubuh menggunakan insulin
Score : 0.1672
------------------------------
Tanya : Bagaimana cara mencegahnya diabetes melitus?
Jawab : kurangnya produksi insulin oleh pankreas atau ketidakmampuan tubuh menggunakan insulin
Score : 0.0686
------------------------------
Tanya : Apa gejala Diabetes melitus?
Jawab : kurangnya produksi insulin oleh pankreas atau ketidakmampuan tubuh menggunakan insulin
Score : 0.1180
------------------------------


# Tahap 5: Search FUnction (_or Search Engine, whaatever you called it_)

In [1]:
import pandas as pd
import torch
from transformers import BertTokenizerFast, BertForQuestionAnswering
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
import chromadb
from chromadb.utils import embedding_functions
import numpy as np
import os

In [2]:
# Load processed chunks
print("📂 Loading dataset...")
df = pd.read_csv('output_dataset/processed_chunks.csv')
print(f"Total chunks: {len(df)}")


📂 Loading dataset...
Total chunks: 7661


In [3]:
# Load QA Model (Reader)
print("\n Loading QA Model...")
qa_model_path = './indobert-gizi-qa-final'
tokenizer = BertTokenizerFast.from_pretrained(qa_model_path)
qa_model = BertForQuestionAnswering.from_pretrained(qa_model_path)
qa_model.to('cuda' if torch.cuda.is_available() else 'cpu')
qa_model.eval()
print(" QA Model loaded")


 Loading QA Model...
 QA Model loaded


In [20]:
# ============================================
# RETRIEVER: Hybrid Search (Dense + BM25)
# ============================================

# Gunakan SentenceTransformer untuk embedding
embedding_model = SentenceTransformer('LazarusNLP/all-indobert-base-v2')

def custom_embedding_function(texts):
    return embedding_model.encode(texts, show_progress_bar=False).tolist()

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

C:\Users\LENOVO\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\LENOVO\.cache\huggingface\hub\models--LazarusNLP--all-indobert-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

In [21]:
# Buat collection
CHROMA_DATA_PATH = "./chroma_db_store" # Folder tempat data akan disimpan selamanya

print("\n🔍 Setting up Retriever...")

# 1. Gunakan PERSISTENT Client (Bukan Client biasa)
print(f"  → Initializing ChromaDB from: {CHROMA_DATA_PATH}")
chroma_client = chromadb.PersistentClient(path=CHROMA_DATA_PATH)

# Setup Embedding Function
embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
)

# 2. Get or Create Collection
# Fungsi ini akan meload collection jika ada, atau membuat baru jika belum ada
collection = chroma_client.get_or_create_collection(
    name="gizi_knowledge",
    embedding_function=embedding_func
)


🔍 Setting up Retriever...
  → Initializing ChromaDB from: ./chroma_db_store


In [22]:
# ============================================
# LOGIKA PENGISIAN DATA CERDAS
# ============================================

# Cek apakah database sudah berisi data?
if collection.count() == 0:
    print("  ⚠️ Database kosong. Memulai proses Indexing Dokumen (Hanya sekali)...")
    
    # --- PROSES ADD DOCUMENT (Sama seperti kode lama Anda) ---
    batch_size = 100
    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i+batch_size]
        collection.add(
            documents=batch['text'].tolist(),
            ids=[f"chunk_{j}" for j in range(i, min(i+batch_size, len(df)))],
            metadatas=[
                {
                    'title': str(row['title']),
                    'url': str(row['url']),
                    'intent': str(row['intent']),
                    'doc_id': int(row['doc_id']),
                    'chunk_id': int(row['chunk_id'])
                }
                for _, row in batch.iterrows()
            ]
        )
        if (i + batch_size) % 500 == 0:
            print(f"    Processed {min(i+batch_size, len(df))}/{len(df)} chunks")
            
    print("  ✅ Indexing Selesai & Tersimpan Otomatis!")

else:
    print(f"  ✅ Database ditemukan berisi {collection.count()} dokumen.")
    print("  ⏩ Skip indexing. Langsung siap dipakai.")

  ✅ Database ditemukan berisi 7661 dokumen.
  ⏩ Skip indexing. Langsung siap dipakai.


In [23]:
# 2. BM25 (Sparse Retrieval)
print("  → Initializing BM25...")
tokenized_corpus = [doc.lower().split() for doc in df['text'].tolist()]
bm25 = BM25Okapi(tokenized_corpus)
print("✅ BM25 ready")

  → Initializing BM25...
✅ BM25 ready


In [24]:
# ============================================
# ENSEMBLE RETRIEVER
# ============================================

def hybrid_retrieve(query, top_k=5, dense_weight=0.6, bm25_weight=0.4):
    """
    Hybrid retrieval: Dense (ChromaDB) + Sparse (BM25)
    """
    # Dense retrieval
    dense_results = collection.query(
        query_texts=[query],
        n_results=top_k * 2
    )
    
    dense_docs = []
    for i, doc_id in enumerate(dense_results['ids'][0]):
        idx = int(doc_id.split('_')[1])
        dense_docs.append({
            'idx': idx,
            'text': dense_results['documents'][0][i],
            'metadata': dense_results['metadatas'][0][i],
            'distance': dense_results['distances'][0][i],
            'dense_score': 1 / (1 + dense_results['distances'][0][i])  # Convert distance to score
        })
    
    # BM25 retrieval
    tokenized_query = query.lower().split()
    bm25_scores = bm25.get_scores(tokenized_query)
    top_bm25_indices = np.argsort(bm25_scores)[::-1][:top_k * 2]
    
    bm25_docs = []
    for idx in top_bm25_indices:
        bm25_docs.append({
            'idx': int(idx),
            'bm25_score': float(bm25_scores[idx])
        })
    
    # Combine scores
    combined = {}
    for doc in dense_docs:
        idx = doc['idx']
        combined[idx] = {
            'text': doc['text'],
            'metadata': doc['metadata'],
            'score': dense_weight * doc['dense_score']
        }
    
    for doc in bm25_docs:
        idx = doc['idx']
        if idx in combined:
            combined[idx]['score'] += bm25_weight * doc['bm25_score']
        else:
            combined[idx] = {
                'text': df.iloc[idx]['text'],
                'metadata': {
                    'title': df.iloc[idx]['title'],
                    'url': df.iloc[idx]['url'],
                    'intent': df.iloc[idx]['intent']
                },
                'score': bm25_weight * doc['bm25_score']
            }
    
    # Sort by combined score
    sorted_results = sorted(combined.items(), key=lambda x: x[1]['score'], reverse=True)
    
    return [
        {
            'text': item[1]['text'],
            'metadata': item[1]['metadata'],
            'score': item[1]['score']
        }
        for item in sorted_results[:top_k]
    ]

In [25]:
def answer_question(question, context, confidence_threshold=0.0): # Set 0.0 biar kita lihat SEMUA hasil
    inputs = tokenizer(
        question,
        context,
        truncation=True,
        max_length=512,
        return_tensors='pt'
    ).to(qa_model.device)
    
    with torch.no_grad():
        outputs = qa_model(**inputs)
    
    start_logits = outputs.start_logits[0]
    end_logits = outputs.end_logits[0]
    
    # Ambil posisi terbaik
    start_idx = torch.argmax(start_logits).item()
    end_idx = torch.argmax(end_logits).item()
    
    # Hitung probabilitas
    start_probs = torch.softmax(start_logits, dim=0)
    end_probs = torch.softmax(end_logits, dim=0)
    confidence = (start_probs[start_idx] * end_probs[end_idx]).item()
    
    # --- BAGIAN DEBUGGING ---
    print(f"\n   🕵️ [DEBUG] Q: {question}")
    print(f"   📍 Start Index: {start_idx} | End Index: {end_idx}")
    print(f"   📊 Confidence: {confidence:.4f}")
    
    # Cek apakah dia menunjuk [CLS] (Index 0)
    if start_idx == 0:
        print("   ❌ Model memprediksi: [CLS] (Unanswerable / Menyerah)")
        return None, confidence

    # Cek apakah start > end (Mustahil)
    if start_idx > end_idx:
        print("   ❌ Model memprediksi: Posisi Terbalik (Start > End)")
        return None, confidence

    # Extract jawaban
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    answer_tokens = tokens[start_idx:end_idx + 1]
    answer = tokenizer.convert_tokens_to_string(answer_tokens).replace('##', '')
    
    print(f"   ✅ Calon Jawaban: {answer}")
    return answer, confidence

In [26]:
# ============================================
# CHATBOT PIPELINE (MODE BYPASS / SEARCH ENGINE)
# ============================================

def chatbot_pipeline(question, top_k=3):
    print(f"\n Question: {question}")
    print("="*60)
    
    # 1. Cari Dokumen (Retriever tetap bekerja!)
    print(" Retrieving relevant documents...")
    retrieved_docs = hybrid_retrieve(question, top_k=top_k)
    
    # Fallback jika tidak ada dokumen
    if not retrieved_docs:
        return {
            'answer': 'Maaf, saya belum menemukan informasi yang relevan di database.',
            'source': None,
            'confidence': 0.0,
            'intent': 'Unknown'
        }
        
    print(f"✅ Found {len(retrieved_docs)} relevant documents")
    
    # 2. BYPASS QA MODEL
    # Alih-alih menyuruh model QA mikir, kita langsung ambil dokumen terbaik (Top 1)
    best_doc = retrieved_docs[0] 
    
    # Ambil snippet (misal 3 kalimat pertama atau 300 karakter)
    full_text = best_doc['text']
    # Potong biar gak kepanjangan
    snippet = full_text[:350] + "..." if len(full_text) > 350 else full_text
    
    # Format jawaban ala Search Engine
    response = (
        f"Berikut informasi yang saya temukan:\n\n"
        f"\"{snippet}\"\n\n"
        f"(Sumber: {best_doc['metadata']['title']})"
    )
    
    return {
        'answer': response,
        'source': best_doc['metadata']['url'],
        'confidence': best_doc['score'], # Pakai skor relevansi pencarian
        'intent': best_doc['metadata']['intent']
    }

In [27]:
# ============================================
# TESTING
# ============================================
# Test queries
test_questions = [
    "Apa penyebab perut buncit?",
    "Bagaimana cara menurunkan berat badan?",
    "Makanan apa yang baik untuk diet?"
]

print("\n Testing pipeline...")
for q in test_questions:
    result = chatbot_pipeline(q)
    print(f"\n Answer: {result['answer']}")
    print(f" Source: {result['source']}")
    print(f" Confidence: {result['confidence']:.2%}")
    print(f"  Intent: {result['intent']}")


 Testing pipeline...

 Question: Apa penyebab perut buncit?
 Retrieving relevant documents...
✅ Found 3 relevant documents

 Answer: Berikut informasi yang saya temukan:

"10 makanan penyebab perut buncit no . 7 sering dikonsumsi . makanan penyebab perut buncit bisa menjadi penyebab penumpukan lemak di perut . kondisi ini bisa meningkatkan berbagai risiko penyakit , namun masih bisa diatasi dengan asupan nutrisi yang tepat , meningkatkan aktivitas fisik , hingga mengurangi stres . lalu , apa saja makanan yang harus d..."

(Sumber: 10 Makanan Penyebab Perut Buncit (No. 7 Sering Dikonsumsi))
 Source: https://doktersehat.com/gaya-hidup/gizi-dan-nutrisi/makanan-penyebab-perut-buncit/
 Confidence: 399.97%
  Intent: berat_badan, Kesehatan_umum, kesehatan_ibu

 Question: Bagaimana cara menurunkan berat badan?
 Retrieving relevant documents...
✅ Found 3 relevant documents

 Answer: Berikut informasi yang saya temukan:

"alasan ketidaksuburan pada perempuan . selain itu , beberapa tikus betina

# Tahap 6: Evaluasi (model gagal)

In [1]:
import json
import pandas as pd
import numpy as np
import os

In [2]:
import torch
from transformers import BertTokenizerFast, BertForQuestionAnswering
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi

In [3]:
import chromadb
from chromadb.utils import embedding_functions
from tqdm import tqdm
import re

In [4]:
# ============================================
# LOAD MODELS & DATA
# ============================================

print("📂 Loading evaluation data...")
# Load test set dari QA dataset
test_data = []
with open('3_qa_dataset_strict_gold.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        test_data.append(json.loads(line))

# Ambil 20% terakhir sebagai test set
split_idx = int(0.8 * len(test_data))
test_data = test_data[split_idx:]
print(f"Test data: {len(test_data)} paragraphs")

# Hitung total QA pairs
total_qa = sum(len(para['qas']) for para in test_data)
print(f"Total QA pairs: {total_qa}")

# Load chunks untuk retrieval
df = pd.read_csv('output_dataset/processed_chunks.csv')

# Load QA Model
print("\n🤖 Loading QA Model...")
qa_model_path = './indobert-gizi-qa-final'
tokenizer = BertTokenizerFast.from_pretrained(qa_model_path)
qa_model = BertForQuestionAnswering.from_pretrained(qa_model_path)
qa_model.to('cuda' if torch.cuda.is_available() else 'cpu')
qa_model.eval()

# Load Retriever
# --- KONFIGURASI PATH ---
# Pastikan path ini SAMA PERSIS dengan tempat Anda menyimpan database sebelumnya
CHROMA_DATA_PATH = "./chroma_db_store" 
COLLECTION_NAME = "gizi_knowledge"

print(f"📂 Memuat ChromaDB Lokal dari: {CHROMA_DATA_PATH} ...")

# 1. Cek apakah folder ada
if not os.path.exists(CHROMA_DATA_PATH):
    print(f"❌ Error: Folder database '{CHROMA_DATA_PATH}' tidak ditemukan.")
    print("   Pastikan Anda sudah menjalankan proses indexing (add documents) setidaknya satu kali.")
else:
    try:
        # 2. Inisialisasi Persistent Client
        chroma_client = chromadb.PersistentClient(path=CHROMA_DATA_PATH)

        # 3. Setup Embedding Function (WAJIB SAMA dengan saat pembuatan)
        # Jika beda model, hasil pencarian akan ngaco.
        embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
            model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
        )

        # 4. Ambil Collection yang Sudah Ada
        # Gunakan get_collection (bukan create_collection) untuk memastikan kita cuma loading
        collection = chroma_client.get_collection(
            name=COLLECTION_NAME,
            embedding_function=embedding_func
        )

        # 5. Verifikasi Isi
        count = collection.count()
        print(f"✅ Berhasil terhubung!")
        print(f"📊 Jumlah Dokumen dalam Database: {count}")
        
        if count == 0:
            print("⚠️ Peringatan: Collection ditemukan tapi KOSONG. Cek proses indexing Anda.")

    except Exception as e:
        print(f"❌ Gagal memuat collection: {e}")
        print("   Tips: Cek apakah nama collection benar atau folder korup.")

# Load BM25
tokenized_corpus = [doc.lower().split() for doc in df['text'].tolist()]
bm25 = BM25Okapi(tokenized_corpus)

📂 Loading evaluation data...
Test data: 151 paragraphs
Total QA pairs: 394

🤖 Loading QA Model...
📂 Memuat ChromaDB Lokal dari: ./chroma_db_store ...
✅ Berhasil terhubung!
📊 Jumlah Dokumen dalam Database: 7661


In [5]:
# ============================================
# RETRIEVAL FUNCTIONS
# ============================================

def hybrid_retrieve(query, top_k=5):
    dense_results = collection.query(query_texts=[query], n_results=top_k * 2)
    
    dense_docs = []
    for i, doc_id in enumerate(dense_results['ids'][0]):
        idx = int(doc_id.split('_')[1])
        dense_docs.append({
            'idx': idx,
            'text': dense_results['documents'][0][i],
            'dense_score': 1 / (1 + dense_results['distances'][0][i])
        })
    
    tokenized_query = query.lower().split()
    bm25_scores = bm25.get_scores(tokenized_query)
    top_bm25_indices = np.argsort(bm25_scores)[::-1][:top_k * 2]
    
    bm25_docs = {int(idx): float(bm25_scores[idx]) for idx in top_bm25_indices}
    
    combined = {}
    for doc in dense_docs:
        idx = doc['idx']
        combined[idx] = {'text': doc['text'], 'score': 0.6 * doc['dense_score']}
    
    for idx, score in bm25_docs.items():
        if idx in combined:
            combined[idx]['score'] += 0.4 * score
        else:
            combined[idx] = {'text': df.iloc[idx]['text'], 'score': 0.4 * score}
    
    sorted_results = sorted(combined.items(), key=lambda x: x[1]['score'], reverse=True)
    return [{'text': item[1]['text'], 'score': item[1]['score']} for item in sorted_results[:top_k]]



In [7]:
# ============================================
# QA FUNCTIONS
# ============================================

def answer_question(question, context):
    inputs = tokenizer(question, context, truncation=True, max_length=512, return_tensors='pt').to(qa_model.device)
    
    with torch.no_grad():
        outputs = qa_model(**inputs)
    
    start_logits = outputs.start_logits[0]
    end_logits = outputs.end_logits[0]
    
    start_probs = torch.softmax(start_logits, dim=0)
    end_probs = torch.softmax(end_logits, dim=0)
    
    start_idx = torch.argmax(start_logits).item()
    end_idx = torch.argmax(end_logits).item()
    
    confidence = (start_probs[start_idx] * end_probs[end_idx]).item()
    
    if start_idx >= end_idx:
        return "", confidence
    
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    answer_tokens = tokens[start_idx:end_idx + 1]
    answer = tokenizer.convert_tokens_to_string(answer_tokens)
    
    return answer, confidence

In [10]:
# ============================================
# EVALUATION METRICS
# ============================================

def normalize_answer(s):
    """Normalisasi teks untuk evaluasi"""
    s = s.lower()
    s = re.sub(r'\s+', ' ', s)
    s = s.strip()
    return s

def compute_exact_match(prediction, ground_truth):
    """Exact Match (EM)"""
    return normalize_answer(prediction) == normalize_answer(ground_truth)

def compute_f1(prediction, ground_truth):
    """F1 Score"""
    pred_tokens = normalize_answer(prediction).split()
    truth_tokens = normalize_answer(ground_truth).split()
    
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common = set(pred_tokens) & set(truth_tokens)
    num_common = len(common)
    
    if num_common == 0:
        return 0
    
    precision = num_common / len(pred_tokens)
    recall = num_common / len(truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    
    return f1

In [11]:
# ============================================
# EVALUATION LOOP
# ============================================

print("\n" + "="*60)
print("📊 EVALUASI CHATBOT")
print("="*60)

results = {
    'retrieval': {'recall@1': [], 'recall@3': [], 'recall@5': []},
    'reader': {'em': [], 'f1': [], 'confidence': []},
    'threshold_analysis': {
        'retriever': {'scores': [], 'has_answer': []},
        'reader': {'scores': [], 'correct': []}
    }
}

print("\n🔍 Evaluating...")
for para in tqdm(test_data, desc="Processing"):
    context = para['context']
    
    for qa in para['qas']:
        question = qa['question']
        is_impossible = qa['is_impossible']
        
        # Skip unanswerable untuk evaluasi QA
        if is_impossible:
            continue
        
        ground_truth = qa['answers'][0]['text']
        
        # === EVALUASI RETRIEVAL ===
        retrieved_docs = hybrid_retrieve(question, top_k=5)
        
        # Check if correct context retrieved
        retrieved_texts = [doc['text'] for doc in retrieved_docs]
        context_found_at = -1
        
        for i, doc_text in enumerate(retrieved_texts):
            if normalize_answer(context) in normalize_answer(doc_text) or \
               normalize_answer(doc_text) in normalize_answer(context):
                context_found_at = i
                break
        
        results['retrieval']['recall@1'].append(1 if context_found_at == 0 else 0)
        results['retrieval']['recall@3'].append(1 if 0 <= context_found_at < 3 else 0)
        results['retrieval']['recall@5'].append(1 if 0 <= context_found_at < 5 else 0)
        
        # Threshold analysis - retriever
        if retrieved_docs:
            results['threshold_analysis']['retriever']['scores'].append(retrieved_docs[0]['score'])
            results['threshold_analysis']['retriever']['has_answer'].append(context_found_at >= 0)
        
        # === EVALUASI READER ===
        # Gunakan context asli untuk evaluasi reader
        prediction, confidence = answer_question(question, context)
        
        # Compute metrics
        em = compute_exact_match(prediction, ground_truth)
        f1 = compute_f1(prediction, ground_truth)
        
        results['reader']['em'].append(em)
        results['reader']['f1'].append(f1)
        results['reader']['confidence'].append(confidence)
        
        # Threshold analysis - reader
        results['threshold_analysis']['reader']['scores'].append(confidence)
        results['threshold_analysis']['reader']['correct'].append(em or f1 > 0.5)


📊 EVALUASI CHATBOT

🔍 Evaluating...


Processing: 100%|████████████████████████████████████████████████████████████████████| 151/151 [00:29<00:00,  5.21it/s]


In [12]:
# ============================================
# COMPUTE FINAL METRICS
# ============================================

print("\n" + "="*60)
print("📈 HASIL EVALUASI")
print("="*60)

print("\n🎯 RETRIEVAL PERFORMANCE:")
print(f"  Recall@1: {np.mean(results['retrieval']['recall@1']):.2%}")
print(f"  Recall@3: {np.mean(results['retrieval']['recall@3']):.2%}")
print(f"  Recall@5: {np.mean(results['retrieval']['recall@5']):.2%}")

print("\n📖 READER PERFORMANCE:")
print(f"  Exact Match (EM): {np.mean(results['reader']['em']):.2%}")
print(f"  F1 Score: {np.mean(results['reader']['f1']):.2%}")
print(f"  Avg Confidence: {np.mean(results['reader']['confidence']):.2%}")


📈 HASIL EVALUASI

🎯 RETRIEVAL PERFORMANCE:
  Recall@1: 32.10%
  Recall@3: 45.68%
  Recall@5: 51.85%

📖 READER PERFORMANCE:
  Exact Match (EM): 19.75%
  F1 Score: 29.54%
  Avg Confidence: 14.39%


In [13]:
# ============================================
# THRESHOLD ANALYSIS
# ============================================

print("\n🎚️ THRESHOLD ANALYSIS:")

# Retriever threshold
retriever_scores = np.array(results['threshold_analysis']['retriever']['scores'])
retriever_has_answer = np.array(results['threshold_analysis']['retriever']['has_answer'])

print("\n  Retriever Score Distribution:")
print(f"    Min: {retriever_scores.min():.4f}")
print(f"    25%: {np.percentile(retriever_scores, 25):.4f}")
print(f"    50%: {np.percentile(retriever_scores, 50):.4f}")
print(f"    75%: {np.percentile(retriever_scores, 75):.4f}")
print(f"    Max: {retriever_scores.max():.4f}")

# Find optimal retriever threshold
thresholds = np.linspace(retriever_scores.min(), retriever_scores.max(), 20)
best_threshold = 0
best_accuracy = 0

for thresh in thresholds:
    passed = retriever_scores >= thresh
    if passed.sum() > 0:
        accuracy = retriever_has_answer[passed].mean()
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_threshold = thresh

print(f"\n  Recommended Retriever Threshold: {best_threshold:.4f}")
print(f"    (Accuracy when passing threshold: {best_accuracy:.2%})")

# Reader threshold
reader_scores = np.array(results['threshold_analysis']['reader']['scores'])
reader_correct = np.array(results['threshold_analysis']['reader']['correct'])

print("\n  Reader Confidence Distribution:")
print(f"    Min: {reader_scores.min():.4f}")
print(f"    25%: {np.percentile(reader_scores, 25):.4f}")
print(f"    50%: {np.percentile(reader_scores, 50):.4f}")
print(f"    75%: {np.percentile(reader_scores, 75):.4f}")
print(f"    Max: {reader_scores.max():.4f}")

# Find optimal reader threshold
best_threshold = 0
best_precision = 0

for thresh in np.linspace(reader_scores.min(), reader_scores.max(), 20):
    passed = reader_scores >= thresh
    if passed.sum() > 0:
        precision = reader_correct[passed].mean()
        if precision > best_precision:
            best_precision = precision
            best_threshold = thresh

print(f"\n  Recommended Reader Threshold: {best_threshold:.4f}")
print(f"    (Precision when passing threshold: {best_precision:.2%})")


🎚️ THRESHOLD ANALYSIS:

  Retriever Score Distribution:
    Min: 2.4894
    25%: 6.2343
    50%: 8.0632
    75%: 10.0758
    Max: 18.5546

  Recommended Retriever Threshold: 16.8635
    (Accuracy when passing threshold: 100.00%)

  Reader Confidence Distribution:
    Min: 0.0070
    25%: 0.0316
    50%: 0.0676
    75%: 0.1515
    Max: 0.9797

  Recommended Reader Threshold: 0.4678
    (Precision when passing threshold: 70.00%)


In [14]:
# ============================================
# SAVE RESULTS
# ============================================

eval_summary = {
    'retrieval': {
        'recall@1': float(np.mean(results['retrieval']['recall@1'])),
        'recall@3': float(np.mean(results['retrieval']['recall@3'])),
        'recall@5': float(np.mean(results['retrieval']['recall@5']))
    },
    'reader': {
        'exact_match': float(np.mean(results['reader']['em'])),
        'f1_score': float(np.mean(results['reader']['f1'])),
        'avg_confidence': float(np.mean(results['reader']['confidence']))
    },
    'recommended_thresholds': {
        'retriever': float(best_threshold),
        'reader': float(best_threshold)
    }
}

with open('evaluation_results.json', 'w', encoding='utf-8') as f:
    json.dump(eval_summary, f, indent=2, ensure_ascii=False)

print("\n💾 Results saved to: evaluation_results.json")
print("="*60)


💾 Results saved to: evaluation_results.json


# Tahap 6: Evaluasi

In [15]:
import json
import pandas as pd
import numpy as np

In [16]:
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
import chromadb
from chromadb.utils import embedding_functions

In [17]:
from tqdm import tqdm
import re

In [18]:
# ============================================
# LOAD MODELS & DATA
# ============================================

print("📂 Loading evaluation data...")
# Load test set dari QA dataset
test_data = []
with open('3_qa_dataset_strict_gold.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        test_data.append(json.loads(line))

# Split 80/20 untuk test
split_idx = int(0.8 * len(test_data))
test_data = test_data[split_idx:]
print(f"Test data: {len(test_data)} paragraphs")

# Hitung total QA pairs
total_qa = sum(len(para['qas']) for para in test_data)
print(f"Total QA pairs: {total_qa}")

# Load chunks untuk retrieval
df = pd.read_csv('output_dataset/processed_chunks.csv')

# Load Retriever
print("\n🔍 Loading Retriever...")
chroma_client = chromadb.PersistentClient(path="./chroma_db_store")
collection = chroma_client.get_collection(
    name="gizi_knowledge",
    embedding_function=embedding_functions.SentenceTransformerEmbeddingFunction(
        model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
    )
)

# Load BM25
tokenized_corpus = [doc.lower().split() for doc in df['text'].tolist()]
bm25 = BM25Okapi(tokenized_corpus)


📂 Loading evaluation data...
Test data: 151 paragraphs
Total QA pairs: 394

🔍 Loading Retriever...


In [19]:
# ============================================
# RETRIEVAL FUNCTIONS
# ============================================

def hybrid_retrieve(query, top_k=5):
    dense_results = collection.query(query_texts=[query], n_results=top_k * 2)
    
    dense_docs = []
    for i, doc_id in enumerate(dense_results['ids'][0]):
        idx = int(doc_id.split('_')[1])
        dense_docs.append({
            'idx': idx,
            'text': dense_results['documents'][0][i],
            'dense_score': 1 / (1 + dense_results['distances'][0][i])
        })
    
    tokenized_query = query.lower().split()
    bm25_scores = bm25.get_scores(tokenized_query)
    top_bm25_indices = np.argsort(bm25_scores)[::-1][:top_k * 2]
    
    bm25_docs = {int(idx): float(bm25_scores[idx]) for idx in top_bm25_indices}
    
    combined = {}
    for doc in dense_docs:
        idx = doc['idx']
        combined[idx] = {'text': doc['text'], 'score': 0.6 * doc['dense_score']}
    
    for idx, score in bm25_docs.items():
        if idx in combined:
            combined[idx]['score'] += 0.4 * score
        else:
            combined[idx] = {'text': df.iloc[idx]['text'], 'score': 0.4 * score}
    
    sorted_results = sorted(combined.items(), key=lambda x: x[1]['score'], reverse=True)
    return [{'text': item[1]['text'], 'score': item[1]['score'], 'idx': item[0]} 
            for item in sorted_results[:top_k]]


In [20]:
# ============================================
# EVALUATION METRICS
# ============================================

def normalize_answer(s):
    """Normalisasi teks untuk evaluasi"""
    s = s.lower()
    s = re.sub(r'\s+', ' ', s)
    s = s.strip()
    return s

def compute_exact_match(prediction, ground_truth):
    """Exact Match (EM)"""
    return int(normalize_answer(prediction) == normalize_answer(ground_truth))

def compute_f1(prediction, ground_truth):
    """F1 Score based on token overlap"""
    pred_tokens = normalize_answer(prediction).split()
    truth_tokens = normalize_answer(ground_truth).split()
    
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common = set(pred_tokens) & set(truth_tokens)
    num_common = len(common)
    
    if num_common == 0:
        return 0
    
    precision = num_common / len(pred_tokens)
    recall = num_common / len(truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    
    return f1

def extract_answer_from_snippet(snippet, ground_truth):
    """
    Cari ground truth dalam snippet
    Jika ada, return True + F1 score
    """
    snippet_norm = normalize_answer(snippet)
    truth_norm = normalize_answer(ground_truth)
    
    # Check if ground truth exist in snippet
    if truth_norm in snippet_norm:
        return True, 1.0  # Perfect match
    
    # Calculate token overlap F1
    f1 = compute_f1(snippet, ground_truth)
    
    # Threshold: F1 > 0.3 dianggap berhasil
    return (f1 > 0.3), f1



In [21]:
# ============================================
# EVALUATION LOOP
# ============================================

print("\n" + "="*60)
print("📊 EVALUASI SEARCH ENGINE")
print("="*60)

results = {
    'retrieval': {
        'recall@1': [],
        'recall@3': [],
        'recall@5': [],
        'mrr': []  # Mean Reciprocal Rank
    },
    'answer_quality': {
        'em': [],
        'f1': [],
        'has_answer': []
    },
    'confidence': {
        'scores': [],
        'correct': []
    }
}

print("\n🔍 Evaluating...")
for para in tqdm(test_data, desc="Processing"):
    context = para['context']
    
    for qa in para['qas']:
        question = qa['question']
        is_impossible = qa['is_impossible']
        
        # Skip unanswerable untuk evaluasi
        if is_impossible:
            continue
        
        ground_truth = qa['answers'][0]['text']
        
        # === EVALUASI RETRIEVAL ===
        retrieved_docs = hybrid_retrieve(question, top_k=5)
        
        # Check if correct context retrieved
        context_found_at = -1
        context_norm = normalize_answer(context)
        
        for i, doc in enumerate(retrieved_docs):
            doc_norm = normalize_answer(doc['text'])
            # Check overlap (flexible matching)
            if context_norm in doc_norm or doc_norm in context_norm:
                context_found_at = i
                break
        
        # Recall@k
        results['retrieval']['recall@1'].append(1 if context_found_at == 0 else 0)
        results['retrieval']['recall@3'].append(1 if 0 <= context_found_at < 3 else 0)
        results['retrieval']['recall@5'].append(1 if 0 <= context_found_at < 5 else 0)
        
        # MRR (Mean Reciprocal Rank)
        if context_found_at >= 0:
            results['retrieval']['mrr'].append(1 / (context_found_at + 1))
        else:
            results['retrieval']['mrr'].append(0)
        
        # === EVALUASI ANSWER QUALITY ===
        # Gunakan top-1 retrieved document sebagai "answer"
        if retrieved_docs:
            top_snippet = retrieved_docs[0]['text'][:400]  # Snippet yang akan ditampilkan
            top_score = retrieved_docs[0]['score']
            
            # Check if ground truth ada di snippet
            has_answer, f1_score = extract_answer_from_snippet(top_snippet, ground_truth)
            
            results['answer_quality']['has_answer'].append(has_answer)
            results['answer_quality']['f1'].append(f1_score)
            
            # EM: strict match (jarang terjadi di snippet mode, tapi tetap dihitung)
            em = compute_exact_match(top_snippet, ground_truth)
            results['answer_quality']['em'].append(em)
            
            # === CONFIDENCE ANALYSIS ===
            results['confidence']['scores'].append(top_score)
            results['confidence']['correct'].append(has_answer)


📊 EVALUASI SEARCH ENGINE

🔍 Evaluating...


Processing: 100%|████████████████████████████████████████████████████████████████████| 151/151 [00:18<00:00,  8.00it/s]


In [22]:
# ============================================
# COMPUTE FINAL METRICS
# ============================================

print("\n" + "="*60)
print("📈 HASIL EVALUASI")
print("="*60)

print("\n🎯 RETRIEVAL PERFORMANCE:")
print(f"  Recall@1: {np.mean(results['retrieval']['recall@1']):.2%}")
print(f"  Recall@3: {np.mean(results['retrieval']['recall@3']):.2%}")
print(f"  Recall@5: {np.mean(results['retrieval']['recall@5']):.2%}")
print(f"  MRR (Mean Reciprocal Rank): {np.mean(results['retrieval']['mrr']):.4f}")

print("\n📖 ANSWER QUALITY (Search Engine Mode):")
print(f"  Answer Found Rate: {np.mean(results['answer_quality']['has_answer']):.2%}")
print(f"  Average F1 Score: {np.mean(results['answer_quality']['f1']):.2%}")
print(f"  Exact Match (EM): {np.mean(results['answer_quality']['em']):.2%}")

print("\n📊 CONFIDENCE SCORE ANALYSIS:")
confidence_scores = np.array(results['confidence']['scores'])
confidence_correct = np.array(results['confidence']['correct'])

print(f"  Min Score: {confidence_scores.min():.4f}")
print(f"  25th Percentile: {np.percentile(confidence_scores, 25):.4f}")
print(f"  Median: {np.percentile(confidence_scores, 50):.4f}")
print(f"  75th Percentile: {np.percentile(confidence_scores, 75):.4f}")
print(f"  Max Score: {confidence_scores.max():.4f}")

# Find optimal threshold
print("\n🎚️ OPTIMAL THRESHOLD ANALYSIS:")
thresholds = np.linspace(confidence_scores.min(), confidence_scores.max(), 20)
best_threshold = 0
best_precision = 0
best_f1 = 0

for thresh in thresholds:
    passed = confidence_scores >= thresh
    if passed.sum() > 0:
        precision = confidence_correct[passed].mean()
        recall = confidence_correct[passed].sum() / len(confidence_correct)
        
        if precision + recall > 0:
            f1 = 2 * precision * recall / (precision + recall)
            if f1 > best_f1:
                best_f1 = f1
                best_threshold = thresh
                best_precision = precision

print(f"  Recommended Threshold: {best_threshold:.4f}")
print(f"    Precision at threshold: {best_precision:.2%}")
print(f"    F1 Score at threshold: {best_f1:.2%}")

# Correlation between confidence and correctness
correlation = np.corrcoef(confidence_scores, confidence_correct)[0, 1]
print(f"  Correlation (Score vs Correctness): {correlation:.4f}")


📈 HASIL EVALUASI

🎯 RETRIEVAL PERFORMANCE:
  Recall@1: 32.10%
  Recall@3: 45.68%
  Recall@5: 51.85%
  MRR (Mean Reciprocal Rank): 0.3965

📖 ANSWER QUALITY (Search Engine Mode):
  Answer Found Rate: 10.70%
  Average F1 Score: 14.59%
  Exact Match (EM): 0.00%

📊 CONFIDENCE SCORE ANALYSIS:
  Min Score: 2.4894
  25th Percentile: 6.2343
  Median: 8.0632
  75th Percentile: 10.0758
  Max Score: 18.5546

🎚️ OPTIMAL THRESHOLD ANALYSIS:
  Recommended Threshold: 7.5626
    Precision at threshold: 16.67%
    F1 Score at threshold: 12.07%
  Correlation (Score vs Correctness): 0.1421


In [23]:
# ============================================
# DETAILED BREAKDOWN
# ============================================

print("\n📋 CONFIDENCE SCORE BUCKETS:")
buckets = [
    (0.0, 0.5, "Very Low"),
    (0.5, 1.0, "Low"),
    (1.0, 1.5, "Medium"),
    (1.5, 2.0, "High"),
    (2.0, 10.0, "Very High")
]

for low, high, label in buckets:
    mask = (confidence_scores >= low) & (confidence_scores < high)
    count = mask.sum()
    if count > 0:
        accuracy = confidence_correct[mask].mean()
        print(f"  {label} ({low:.1f}-{high:.1f}): {count} samples, Accuracy: {accuracy:.2%}")


📋 CONFIDENCE SCORE BUCKETS:
  Very High (2.0-10.0): 180 samples, Accuracy: 8.33%


In [24]:

# ============================================
# SAVE RESULTS
# ============================================

eval_summary = {
    'retrieval': {
        'recall@1': float(np.mean(results['retrieval']['recall@1'])),
        'recall@3': float(np.mean(results['retrieval']['recall@3'])),
        'recall@5': float(np.mean(results['retrieval']['recall@5'])),
        'mrr': float(np.mean(results['retrieval']['mrr']))
    },
    'answer_quality': {
        'answer_found_rate': float(np.mean(results['answer_quality']['has_answer'])),
        'average_f1': float(np.mean(results['answer_quality']['f1'])),
        'exact_match': float(np.mean(results['answer_quality']['em']))
    },
    'confidence': {
        'min': float(confidence_scores.min()),
        'median': float(np.median(confidence_scores)),
        'max': float(confidence_scores.max()),
        'correlation': float(correlation),
        'optimal_threshold': float(best_threshold),
        'precision_at_threshold': float(best_precision)
    }
}

with open('evaluation_results.json', 'w', encoding='utf-8') as f:
    json.dump(eval_summary, f, indent=2, ensure_ascii=False)

print("\n💾 Results saved to: evaluation_results.json")
print("="*60)


💾 Results saved to: evaluation_results.json
